In [21]:
import os

In [22]:
%pwd

'c:\\Users\\besec\\OneDrive\\Desktop'

In [33]:

os.chdir(r"C:\Users\besec\OneDrive\Desktop\MLOPS-Project-")



In [34]:
%pwd

'C:\\Users\\besec\\OneDrive\\Desktop\\MLOPS-Project-'

In [35]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [26]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf
# ✅ Ensure eager execution is enabled
tf.config.run_functions_eagerly(True)
tf.compat.v1.enable_eager_execution()

In [27]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "kidney-ct-scan-image")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [28]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [36]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    def get_base_model(self):
        """
        Load the saved model but avoid restoring the old optimizer state.
        Then create a new optimizer instance and compile the model for further training.
        """
        try:
            # Attempt to load model (use str path). Do NOT restore old optimizer state -> compile=False
            self.model = tf.keras.models.load_model(
                str(self.config.updated_base_model_path),
                compile=False
            )
        except TypeError as e:
            # If we hit the InputLayer 'batch_shape' problem from earlier, use patched InputLayer
            if "Unrecognized keyword arguments: ['batch_shape']" in str(e):
                class PatchedInputLayer(tf.keras.layers.InputLayer):
                    @classmethod
                    def from_config(cls, config):
                        if 'batch_shape' in config and 'batch_input_shape' not in config:
                            config = dict(config)
                            config['batch_input_shape'] = config.pop('batch_shape')
                        return super().from_config(config)

                self.model = tf.keras.models.load_model(
                    str(self.config.updated_base_model_path),
                    custom_objects={"InputLayer": PatchedInputLayer},
                    compile=False
                )
            else:
                raise

        # Create a new optimizer instance (fresh, not reused)
        new_optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)

        # Decide loss depending on generator class_mode (default to categorical)
        loss = "categorical_crossentropy"
        try:
            cm = getattr(self, "train_generator", None)
            if cm is not None and getattr(cm, "class_mode", None) is not None:
                cmode = cm.class_mode
                if cmode == "binary":
                    loss = "binary_crossentropy"
                elif cmode == "sparse":
                    loss = "sparse_categorical_crossentropy"
                else:
                    loss = "categorical_crossentropy"
        except Exception:
            loss = "categorical_crossentropy"

        # Compile model with new optimizer and chosen loss
        self.model.compile(optimizer=new_optimizer, loss=loss, metrics=["accuracy"])

        print(f"Loaded model from {self.config.updated_base_model_path} and compiled with optimizer={type(new_optimizer).__name__}, loss={loss}")

    def train_valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1.0 / 255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [37]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2025-10-17 21:12:39,315: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-10-17 21:12:39,322: INFO: common: yaml file: params.yaml loaded successfully]
[2025-10-17 21:12:39,326: INFO: common: created directory at: artifacts]
[2025-10-17 21:12:39,328: INFO: common: created directory at: artifacts\training]
Loaded model from artifacts\prepare_base_model\base_model_updated.h5 and compiled with optimizer=Adam, loss=categorical_crossentropy
Found 93 images belonging to 2 classes.
Found 372 images belonging to 2 classes.


c:\Users\besec\anaconda3\envs\my_env3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
c:\Users\besec\anaconda3\envs\my_env3\Lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


23/23 ━━━━━━━━━━━━━━━━━━━━ 261s 11s/step - accuracy: 0.7135 - loss: 0.5691 - val_accuracy: 0.4375 - val_loss: 0.9809
[2025-10-17 21:17:01,957: WARNING: saving_api: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]
